# Solución Caso 3

## Conjuntos Necesarios

In [1]:
import pandas as pd
#Clientes y sus datos
clients_df = pd.read_csv('data/Clients.csv')
clients_dict = {f"C{int(row['ClientID'])}": [int(row['Product']),row['Longitude'],row['Latitude']] for _, row in clients_df.iterrows()}
print(clients_dict)

#Depositos y sus datos
deposits_df = pd.read_csv('data/Depots_With_Products.csv')
deposits_dict = {f"D{int(row['DepotID'])}": [int(row['DepotID']),row['Longitude'],row['Latitude'], int(row['Product'])] for _, row in deposits_df.iterrows()}
print(deposits_dict)

#Vehículos y sus datos
vehicles_df = pd.read_csv('data/Vehicles.csv')
type_mapping = {
    'gas car': 'GS',
    'ev': 'EV',
    'solar ev': 'EV',
    'drone': 'DR'
}
vehicles_dict = {}
for idx, row in enumerate(vehicles_df.itertuples(), start=1):
    vehicle_id = f"V{idx}"  # Claves como V1, V2, ...
    vehicles_dict[vehicle_id] = [type_mapping[row.VehicleType.lower()], row.Capacity, row.Range]
print(vehicles_dict)

#Datos de Costos por tipo de vehículo
vehicles_data_df = pd.read_csv('data/vehicles_data.csv')
vehicles_data_df.rename(
    columns={
        'Vehicle': 'Type',
        'Freight Rate [COP/km]': 'TF',
        'Time Rate [COP/min]': 'TT',
        'Daily Maintenance [COP/day]': 'CM',
        'Recharge/Fuel Cost [COP/(gal or kWh)]': 'CRC',
        'Recharge/Fuel Time [min/10 percent charge]': 'TR',
        'Avg. Speed [km/h]': 'VP',
        'Gas Efficiency [km/gal]': 'Gas_Efficiency',
        'Electricity Efficency [kWh/km]': 'Electricity_Efficiency'
    },
    inplace=True
)

vehicles_parameters = {}
for _, row in vehicles_data_df.iterrows():
    vehicle_type = type_mapping[row['Type'].lower()]
    vehicles_parameters[vehicle_type] = {
        'TF': row['TF'],
        'TT': row['TT'],
        'CM': row['CM'],
        'CRC': row['CRC'],
        'TR': row['TR'],
        'VP': row['VP'],
        'EC': row['Gas_Efficiency'] if not pd.isna(row['Gas_Efficiency']) else row['Electricity_Efficiency']
    }
print(vehicles_parameters)

#Matriz de distancias terrestres
distance_matrix_df = pd.read_csv('data/distance_matrix.csv', index_col=0)
# distance_matrix_df = pd.read_csv('distance_single_matrix.csv', index_col=0)
distance_dict = {
    (row, col): float(distance_matrix_df.loc[row, col])
    for row in distance_matrix_df.index
    for col in distance_matrix_df.columns
}
print({key: distance_dict[key] for key in list(distance_dict.keys())[:20]})

#Matriz de tiempos terrestres
duration_matrix_df = pd.read_csv('data/duration_matrix.csv', index_col=0)
# duration_matrix_df = pd.read_csv('duration_single_matrix.csv', index_col=0)
duration_dict = {
    (row, col): float(duration_matrix_df.loc[row, col])
    for row in duration_matrix_df.index
    for col in duration_matrix_df.columns
}
print({key: duration_dict[key] for key in list(duration_dict.keys())[:20]})

#Matriz de distancias aéreas
distance_air_df = pd.read_csv('data/distance_dron_matrix.csv', index_col=0)
# distance_air_df = pd.read_csv('distance_dron_single_matrix.csv', index_col=0)
distance_air_dict = {
    (row, col): float(distance_air_df.loc[row, col])
    for row in distance_air_df.index
    for col in distance_air_df.columns
}
print({key: distance_air_dict[key] for key in list(distance_air_dict.keys())[:20]})

{'C1': [12, -74.19699184741948, 4.632552840424734], 'C2': [15, -74.15503659129905, 4.601328214582278], 'C3': [15, -74.10178731827096, 4.732421040989568], 'C4': [6, -74.19486224157397, 4.638612189685843], 'C5': [5, -74.11027242131048, 4.727691608175209], 'C6': [11, -74.15228946899238, 4.665002925306577], 'C7': [12, -74.03241075805882, 4.6771016997028205], 'C8': [10, -74.0624758742746, 4.707007012114083], 'C9': [15, -74.09804168568287, 4.636074548012315]}
{'D1': [1, -74.08124218159384, 4.75021190869025, 8], 'D2': [2, -74.10993358606953, 4.5363832206427785, 10], 'D3': [3, -74.03854814565923, 4.792925960208614, 0], 'D4': [4, -74.06706883098641, 4.72167778077445, 4], 'D5': [5, -74.13826337931849, 4.607707046760958, 28], 'D6': [6, -74.12400186370824, 4.650463053612691, 3], 'D7': [7, -74.09561875464892, 4.621911772492814, 0], 'D8': [8, -74.10975623736951, 4.678960680833056, 10], 'D9': [9, -74.09547235719887, 4.735973062153282, 43], 'D10': [10, -74.10991610076434, 4.550640992537941, 1], 'D11':

## Modelo

In [2]:
from pyomo.environ import *

model = ConcreteModel()

# Conjuntos
unique_nodes = set(i for i, j in distance_dict.keys()).union(j for i, j in distance_dict.keys())
model.I = Set(initialize=unique_nodes)  # Nodos (orígenes y destinos)
model.PAIRS = Set(dimen=2, initialize=distance_dict.keys())  # Pares válidos (i, j)
model.K = Set(initialize=vehicles_dict.keys())  # Vehículos
model.D = Set(initialize=deposits_dict.keys())  # Depósitos




# Parámetros
# Distancia y tiempo entre nodos
model.distance = Param(model.PAIRS, initialize=distance_dict)
model.duration = Param(model.PAIRS, initialize=duration_dict)
model.distance_air = Param(model.PAIRS, initialize=distance_air_dict)

# Parámetros del vehículo
model.TF = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TF'] for k in model.K})
model.TT = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TT'] for k in model.K})
model.CM = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['CM'] for k in model.K})
model.CRC = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['CRC'] for k in model.K})
model.TR = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TR'] for k in model.K})
model.VP = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['VP'] for k in model.K})
model.EC = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['EC'] for k in model.K})
model.R = Param(model.K, initialize={k: vehicles_dict[k][2] for k in model.K})


# Variables de decisión
model.x = Var(model.I, model.I, model.K, domain=Binary)  # Si el vehículo k viaja de i a j
model.W = Var(model.K, domain=NonNegativeReals)  # Peso total transportado por vehículo k
model.u = Var(model.I, model.K, domain=NonNegativeReals)  # Posición secuencial en la ruta para evitar subtours

# Variables auxiliares
model.s = Var(model.K, model.D, domain=Binary)
model.W_s = Var(model.K, model.D, domain=NonNegativeReals)


# Función objetivo
def total_cost_rule(model):
    # Costo por distancia (calculado desde los recorridos)
    C_total = sum(
        model.TF[k] * (
            model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]
        ) * model.x[i, j, k]
        for k in model.K for i in model.I for j in model.I if i != j
    )
    
    # Costo por tiempo
    C_tiempo = sum(
    model.TT[k] * (
        (model.distance[i, j] / 1000 / (model.VP[k] / 60)) if vehicles_dict[k][0] == 'DR' else (model.duration[i, j] / 60)
    ) * model.x[i, j, k]
    for k in model.K for i in model.I for j in model.I if i != j)

    # Costo por peso transportado
    C_carga = sum(model.W[k] * 500 for k in model.K)
    
    # Costo de mantenimiento diario
    C_mantenimiento = sum(
    model.CM[k] * sum(
        model.x[d, j, k] for d in deposits_dict.keys() for j in model.I if d != j
    )
    for k in model.K
)

    # Costo total
    return C_total + C_tiempo + C_carga + C_mantenimiento

model.obj = Objective(rule=total_cost_rule, sense=minimize)

# Restricciones

# Salida desde un depósito
def salida_desde_deposito_rule(model, k):
    return sum(model.x[d, j, k] for d in deposits_dict.keys() for j in model.I if d != j) <= 1
model.salida_desde_deposito = Constraint(model.K, rule=salida_desde_deposito_rule)

# Regreso al depósito de origen
def regreso_a_deposito_rule(model, k):
    return sum(model.x[j, d, k] for d in deposits_dict.keys() for j in model.I if d != j) == sum(model.x[d, j, k] for d in deposits_dict.keys() for j in model.I if d != j)
model.regreso_a_deposito = Constraint(model.K, rule=regreso_a_deposito_rule)

# Regreso al mismo depósito de origen
def regreso_al_mismo_deposito_rule(model, k, d):
    if d in deposits_dict.keys():
        # Si el vehículo sale del depósito `d`, debe regresar a él
        return sum(model.x[d, j, k] for j in model.I if d != j) == sum(model.x[j, d, k] for j in model.I if d != j)
    else:
        return Constraint.Skip  # Ignorar para nodos que no son depósitos
# model.regreso_al_mismo_deposito = Constraint(model.K, deposits_dict.keys(), rule=regreso_al_mismo_deposito_rule)

# Flujo correcto entre nodos (clientes y depósitos)
def flujo_correcto_rule(model, i, k):
    if i in deposits_dict.keys():  # No aplica para depósitos
        return Constraint.Skip
    return sum(model.x[i, j, k] for j in model.I if i != j) == sum(model.x[j, i, k] for j in model.I if i != j)
model.flujo_correcto = Constraint(model.I, model.K, rule=flujo_correcto_rule)

# MTZ para evitar subtours
def mtz_rule(model, i, j, k):
    if i == j or i in deposits_dict.keys() or j in deposits_dict.keys():
        return Constraint.Skip
    n = len(model.I)
    return model.u[i, k] - model.u[j, k] + (n - 1) * model.x[i, j, k] <= n - 2
model.mtz = Constraint(model.I, model.I, model.K, rule=mtz_rule)

# Capacidad Máxima de Carga
def capacidad_carga_rule(model, k):
    return model.W[k] <= vehicles_dict[k][1]  # Capacidad máxima del vehículo k
model.capacidad_carga = Constraint(model.K, rule=capacidad_carga_rule)

# Demanda de los clientes
def demanda_clientes_rule(model, i):
    if i not in clients_dict.keys():
        return Constraint.Skip
    return sum(model.x[j, i, k] for k in model.K for j in model.I if j != i) >= 1
model.demanda_clientes = Constraint(model.I, rule=demanda_clientes_rule)

# Capacidad y Demanda
def capacidad_demanda_rule(model, k):
    return model.W[k] == sum(clients_dict[i][0] * sum(model.x[i, j, k] for j in model.I if i != j) for i in clients_dict.keys())
model.capacidad_demanda = Constraint(model.K, rule=capacidad_demanda_rule)

# Distancia Máxima (Rango)
def distancia_maxima_rule(model, k):
    return sum(
        (model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]) * model.x[i, j, k]
        for i in model.I for j in model.I if i != j
    ) <= vehicles_dict[k][2]
model.distancia_maxima = Constraint(model.K, rule=distancia_maxima_rule)

# Relacionar s[k,d] con x[d,j,k]
def s_definition_rule(model, k, d):
    return model.s[k, d] == sum(model.x[d, j, k] for j in model.I if (d, j, k) in model.x)
model.s_definition = Constraint(model.K, model.D, rule=s_definition_rule)

# Asegurar que cada vehículo sale de un único depósito
def unica_salida_deposito_rule(model, k):
    return sum(model.s[k, d] for d in model.D) <= 1
model.unica_salida_deposito = Constraint(model.K, rule=unica_salida_deposito_rule)

# Linearización del producto W[k] * s[k,d]
def linearizacion_producto_rule1(model, k, d):
    return model.W_s[k, d] <= model.W[k]
model.linearizacion_producto1 = Constraint(model.K, model.D, rule=linearizacion_producto_rule1)

def linearizacion_producto_rule2(model, k, d):
    return model.W_s[k, d] <= vehicles_dict[k][1] * model.s[k, d]
model.linearizacion_producto2 = Constraint(model.K, model.D, rule=linearizacion_producto_rule2)

def linearizacion_producto_rule3(model, k, d):
    return model.W_s[k, d] >= model.W[k] - vehicles_dict[k][1] * (1 - model.s[k, d])
model.linearizacion_producto3 = Constraint(model.K, model.D, rule=linearizacion_producto_rule3)

# Capacidad del depósito
def capacidad_deposito_rule(model, d):
    return sum(model.W_s[k, d] for k in model.K) <= deposits_dict[d][3]
model.capacidad_deposito = Constraint(model.D, rule=capacidad_deposito_rule)

(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo


In [3]:
solver_name = "appsi_highs"
solver = SolverFactory(solver_name)
solver.options['parallel'] = 'on'
solver.options['threads'] = 6
solver.options['time_limit'] = 3600  # 1-hour time limit
solver.options['mip_rel_gap'] = 0.05  # 5% relative gap
result = solver.solve(model, tee=True)

Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [7e-01, 1e+02]
  Cost   [5e+02, 3e+05]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+03]
Presolving model
799 rows, 2508 cols, 11840 nonzeros  0s
776 rows, 1210 cols, 6407 nonzeros  0s
627 rows, 1105 cols, 5830 nonzeros  0s

Solving MIP model with:
   627 rows
   1105 cols (1003 binary, 0 integer, 6 implied int., 96 continuous)
   5830 nonzeros
MIP-Timing:        0.27 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol         

## Resultados

### Archivo CSV Rutas

In [4]:
import csv

# Ruta del archivo CSV de resultados
csv_results_path = "results/reporte_rutas.csv"

# Crear el archivo CSV
with open(csv_results_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Escribir encabezados
    writer.writerow(["Vehículo", "Ruta", "Distancia Total (km)", "Carga Total (kg)"])

    # Generar rutas para cada vehículo
    for k in model.K:
        # Reconstruir la ruta ordenada
        ruta = []
        nodos_visitados = set()
        actual = None

        # Buscar el depósito de inicio
        for d in deposits_dict.keys():
            for j in model.I:
                if d != j and model.x[d, j, k].value > 0.5:
                    ruta.append(d)
                    actual = j
                    break
            if actual:
                break  # Salir si encontramos el nodo inicial

        # Seguir la ruta
        while actual and actual not in nodos_visitados:
            nodos_visitados.add(actual)
            ruta.append(actual)
            siguiente = None
            for j in model.I:
                if actual != j and model.x[actual, j, k].value > 0.5:
                    siguiente = j
                    break
            actual = siguiente

        # Agregar el depósito de fin si existe una conexión
        for d in deposits_dict.keys():
            if actual == d:
                ruta.append(d)
                break

        # Calcular distancia total y peso total transportado
        if len(ruta) > 1:  # Asegurarse de que hay al menos una ruta válida
            distancia_total = sum(
                (model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]) * model.x[i, j, k].value
                for i, j in zip(ruta[:-1], ruta[1:])
            )
        else:
            distancia_total = 0.0

        carga_total = model.W[k].value if ruta else 0.0

        # Escribir la ruta al CSV
        writer.writerow([k, " -> ".join(ruta) if ruta else "Sin ruta", f"{distancia_total:.2f}", f"{carga_total:.2f}"])

print(f"Archivo CSV de rutas generado: {csv_results_path}")


Archivo CSV de rutas generado: results/reporte_rutas.csv


### Reporte Resultados

In [5]:
results_path = "results/reporte_resultados.txt"

with open(results_path, "w") as file:
    # Escribir el costo total
    file.write("=== Resultados ===\n")
    file.write(f"Costo total operativo: {model.obj():.2f} COP\n\n")

    # Escribir los recorridos por vehículo en orden
    file.write("Recorridos asignados por vehículo:\n")
    for k in model.K:
        file.write(f"\nVehículo {k}:\n")
        ruta = []
        nodos_visitados = set()
        actual = None

        # Buscar el depósito de inicio
        for d in deposits_dict.keys():
            for j in model.I:
                if d != j and model.x[d, j, k].value > 0.5:
                    ruta.append(d)
                    actual = j
                    break
            if actual:
                break  # Salir si se encuentra un nodo inicial

        # Seguir la ruta
        while actual and actual not in nodos_visitados:
            nodos_visitados.add(actual)
            ruta.append(actual)
            siguiente = None
            for j in model.I:
                if actual != j and model.x[actual, j, k].value > 0.5:
                    siguiente = j
                    break
            actual = siguiente

        # Agregar el depósito de fin si existe una conexión
        for d in deposits_dict.keys():
            if actual == d:
                ruta.append(d)
                break

        # Escribir la ruta ordenada
        if ruta:
            file.write(f"  Ruta: {' -> '.join(ruta)}\n")
        else:
            file.write("  No tiene asignaciones.\n")

    # Escribir distancias totales recorridas
    file.write("\nDistancias totales recorridas:\n")
    for k in model.K:
        ruta = []
        nodos_visitados = set()
        actual = None

        # Buscar el depósito de inicio
        for d in deposits_dict.keys():
            for j in model.I:
                if d != j and model.x[d, j, k].value > 0.5:
                    ruta.append(d)
                    actual = j
                    break
            if actual:
                break  # Salir si se encuentra un nodo inicial

        # Seguir la ruta
        while actual and actual not in nodos_visitados:
            nodos_visitados.add(actual)
            ruta.append(actual)
            siguiente = None
            for j in model.I:
                if actual != j and model.x[actual, j, k].value > 0.5:
                    siguiente = j
                    break
            actual = siguiente

        # Agregar el depósito de fin si existe una conexión
        for d in deposits_dict.keys():
            if actual == d:
                ruta.append(d)
                break
        if len(ruta) > 1:  # Asegurarse de que hay al menos una ruta válida
            distancia_total = sum(
                (model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]) * model.x[i, j, k].value
                for i, j in zip(ruta[:-1], ruta[1:])
            )
        else:
            distancia_total = 0.0

        carga_total = model.W[k].value if ruta else 0.0
        file.write(f"  Vehículo {k}: {distancia_total:.2f} km\n")

    # Escribir pesos totales transportados
    file.write("\nPesos totales transportados:\n")
    for k in model.K:
        file.write(f"  Vehículo {k}: {model.W[k].value:.2f} kg\n")

print(f"Resultados escritos en el archivo: {results_path}")


Resultados escritos en el archivo: results/reporte_resultados.txt


### Mapa Interactivo

In [6]:
import folium
from folium.plugins import AntPath
import requests
from folium import FeatureGroup, LayerControl

# Crear un mapa centrado en una ubicación aproximada
centro_lat = deposits_df['Latitude'].mean()
centro_lon = deposits_df['Longitude'].mean()
mapa = folium.Map(location=[centro_lat, centro_lon], zoom_start=12)

# Asignar colores según el tipo de vehículo
tipo_colores = {
    "GS": "red",  # Gasoline cars
    "EV": "blue",  # Electric vehicles
    "DR": "green"  # Drones
}

colors = ['red', 'blue', 'green', 'purple', 'orange', 'black', 'darkred', 'darkblue']
vehicle_colors = {f"V{idx+1}": colors[idx % len(colors)] for idx in range(len(model.K))}

# Crear un grupo de características para cada vehículo
vehicle_groups = {k: FeatureGroup(name=f"Vehículo {k}") for k in model.K}


# Agregar marcadores para depósitos
for depot_id, (id, lon, lat, prod) in deposits_dict.items():
    folium.Marker(
        location=[lat, lon],
        popup=f"Depósito {depot_id} - Capacidad {prod} kg",
        icon=folium.Icon(color="darkblue", icon="warehouse", prefix="fa")
    ).add_to(mapa)

# Agregar marcadores para clientes
for client_id, (product, lon, lat) in clients_dict.items():
    folium.Marker(
        location=[lat, lon],
        popup=f"Cliente {client_id} - Demanda {product}",
        icon=folium.Icon(color="green", icon="user", prefix="fa")
    ).add_to(mapa)

# Función para calcular ruta real con OSRM
def calcular_ruta_osrm(coord_inicio, coord_fin):
    try:
        url = f"http://router.project-osrm.org/route/v1/driving/{coord_inicio[1]},{coord_inicio[0]};{coord_fin[1]},{coord_fin[0]}"
        params = {"overview": "full", "geometries": "geojson"}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            route = response.json()
            geometry = route["routes"][0]["geometry"]["coordinates"]
            return [(lat, lon) for lon, lat in geometry]
        else:
            print(f"Error OSRM: {response.status_code}")
            return [coord_inicio, coord_fin]
    except Exception as e:
        print(f"Error al calcular ruta OSRM: {e}")
        return [coord_inicio, coord_fin]

# Dibujar los recorridos de los vehículos con estilos según el tipo
# Dibujar los recorridos de los vehículos con control de capas
for k in model.K:
    color = vehicle_colors[k]  # Asignar color único al vehículo
    grupo = vehicle_groups[k]

    for i in model.I:
        for j in model.I:
            if i != j and model.x[i, j, k].value > 0.5:  # Si hay un recorrido entre i y j
                # Obtener coordenadas de los nodos
                if i in clients_dict:
                    coord_i = (clients_dict[i][2], clients_dict[i][1])  # (lat, lon)
                else:
                    coord_i = (deposits_dict[i][2], deposits_dict[i][1])  # (lat, lon)
                if j in clients_dict:
                    coord_j = (clients_dict[j][2], clients_dict[j][1])  # (lat, lon)
                else:
                    coord_j = (deposits_dict[j][2], deposits_dict[j][1])  # (lat, lon)

                # Calcular la ruta (línea recta para drones, ruta OSRM para otros)
                ruta = [coord_i, coord_j] if vehicles_dict[k][0] == "DR" else calcular_ruta_osrm(coord_i, coord_j)

                # Dibujar la línea
                AntPath(
                    locations=ruta,
                    color=color,
                    weight=2.5,
                    opacity=1,
                    dash_array=[1, 20],
                    tooltip=(f"Vehículo {k}: {i} -> {j}<br>"
                             f"Distancia: {model.distance[i, j] / 1000:.2f} km<br>"
                             f"Peso transportado: {model.W[k].value:.2f} kg")
                ).add_to(grupo)

    # Agregar el grupo al mapa
    grupo.add_to(mapa)

# Agregar el control de capas
LayerControl(collapsed=False).add_to(mapa)


# Guardar el mapa en un archivo HTML
mapa.save("results/mapa_rutas.html")
print("Mapa guardado como 'mapa_rutas.html'")


Mapa guardado como 'mapa_rutas.html'


### PDF de Reporte

In [7]:
%pip install fpdf

Note: you may need to restart the kernel to use updated packages.


In [8]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, 'Informe de Rutas Optimizadas', border=0, ln=True, align='C')
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 10)
        self.cell(0, 10, f'Página {self.page_no()}', align='C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, ln=True, align='L')
        self.ln(5)

    def chapter_body(self, body):
        self.set_font('Arial', '', 11)
        self.multi_cell(0, 10, body)
        self.ln()

    def add_table(self, header, data, col_widths):
        self.set_font('Arial', 'B', 11)
        for i, col in enumerate(header):
            self.cell(col_widths[i], 10, col, border=1, align='C')
        self.ln()
        self.set_font('Arial', '', 10)
        for row in data:
            for i, item in enumerate(row):
                if isinstance(item, str) and item.startswith("http"):  # Enlace clickeable
                    self.set_text_color(0, 0, 255)
                    self.cell(col_widths[i], 10, 'Ver recorrido', border=1, align='C', link=item)
                    self.set_text_color(0, 0, 0)
                else:
                    self.cell(col_widths[i], 10, str(item), border=1, align='C')
            self.ln()

# Crear el PDF
pdf = PDF()
pdf.add_page()

# Título del informe
pdf.set_font('Arial', 'B', 16)
pdf.cell(0, 10, 'Resultados del Modelo de Optimización', align='C', ln=True)
pdf.ln(10)

# Resumen general
pdf.chapter_title('Resumen de Resultados:')
summary = (
    "Costo total operativo: 807,789.26 COP\n\n"
    "Distancias totales recorridas (km):\n"
    "  - Vehículo V5: 21.45 km\n"
    "  - Vehículo V6: 35.14 km\n"
    "  - Vehículo V7: 20.10 km\n"
    "  - Vehículo V8: 7.84 km\n"
    "  - Vehículo V9: 9.12 km\n"
    "  - Vehículo V10: 2.10 km\n"
    "  - Vehículo V11: 14.09 km\n"
    "  - Vehículo V12: 2.85 km\n"
)
pdf.chapter_body(summary)

# Información de vehículos
pdf.chapter_title('Pesos Totales Transportados por Vehículo (kg):')
header = ['Vehículo', 'Peso Transportado']
data = [
    ['V5', '77.00'], ['V6', '78.00'], ['V7', '69.00'],
    ['V8', '36.00'], ['V9', '45.00'], ['V10', '18.00'],
    ['V11', '32.00'], ['V12', '22.00']
]
pdf.add_table(header, data, [50, 50])

# Rutas por vehículo con un enlace único por recorrido
pdf.chapter_title('Recorridos Asignados por Vehículo (Enlace único):')
for vehicle, path in routes.items():
    # Construir la lista de coordenadas en orden
    coordinates = []
    for start, end in path:
        if not coordinates or coordinates[-1] != start:
            coord_start = (clients_dict[start][2], clients_dict[start][1]) if start in clients_dict else (deposits_dict[start][2], deposits_dict[start][1])
            coordinates.append(coord_start)
        coord_end = (clients_dict[end][2], clients_dict[end][1]) if end in clients_dict else (deposits_dict[end][2], deposits_dict[end][1])
        coordinates.append(coord_end)
    
    # Crear el enlace de Google Maps con múltiples paradas
    base_url = "https://www.google.com/maps/dir/"
    map_link = base_url + "/".join([f"{lat:.6f},{lon:.6f}" for lat, lon in coordinates])

    # Agregar el vehículo y su enlace al PDF
    pdf.chapter_title(f'Vehículo {vehicle} ({vehicles_dict[vehicle][0]}):')
    pdf.chapter_body(f"Enlace a la ruta completa: ")
    pdf.cell(0, 10, 'Ver recorrido', ln=True, link=map_link)

# Guardar el PDF
pdf_path = "results/informe_rutas_viaje_unico.pdf"
pdf.output(pdf_path)
print(f"Informe generado: {pdf_path}")


NameError: name 'routes' is not defined